# Cohere Reranker로 검색 결과 재정렬하기

이 노트북에서는 Cohere의 재정렬(reranking) 모델을 사용하여 검색 결과의 품질을 향상시키는 방법을 학습합니다.

## 학습 목표
- Cohere의 다국어 임베딩 모델 이해
- Cohere Reranker를 활용한 검색 결과 재정렬
- ContextualCompressionRetriever 활용법

## 주요 개념
- **Reranking**: 초기 검색 결과를 재정렬하여 가장 관련성 높은 문서를 상위에 배치
- **Contextual Compression**: 쿼리와 관련된 문서만 선택하고 압축

## 1. Cohere 소개

>[Cohere](https://cohere.ai/about)는 기업이 인간-기계 상호작용을 개선할 수 있도록 돕는 자연어 처리 모델을 제공하는 캐나다의 스타트업입니다.

### Cohere 다국어 지원 모델

**Embedding 모델:**
- `embed-multilingual-v3.0`: 최신 다국어 임베딩 모델
- `embed-multilingual-light-v3.0`: 경량화된 다국어 임베딩 모델
- `embed-multilingual-v2.0`: 이전 버전 다국어 임베딩 모델

**Reranker 모델:**
- `rerank-multilingual-v3.0`: 최신 다국어 재정렬 모델
- `rerank-multilingual-v2.0`: 이전 버전 다국어 재정렬 모델

## 2. 환경 설정

In [ ]:
# 필요한 패키지 설치
# !pip install langchain langchain-community langchain-cohere python-dotenv faiss-cpu

In [ ]:
# API KEY를 환경변수로 관리하기 위한 설정
from dotenv import load_dotenv
import os

# .env 파일에서 API KEY 정보 로드
load_dotenv()

# Cohere API 키 확인 (선택사항)
# print("Cohere API Key loaded:", "COHERE_API_KEY" in os.environ)

## 3. 유틸리티 함수 정의

In [1]:
def pretty_print_docs(docs):
    """검색된 문서를 보기 좋게 출력하는 함수"""
    print(
        f"\n{'-' * 100}\n".join(
            [f"Document {i+1}:\n\n{d.page_content}" 
             for i, d in enumerate(docs)]
        )
    )

## 4. 기본 검색기 구성

먼저 문서를 로드하고, 분할한 후 FAISS 벡터 스토어에 저장합니다.

In [2]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_cohere import CohereEmbeddings

# 문서 로드 (예제 파일 경로를 실제 파일로 변경해주세요)
documents = TextLoader("../appendix-keywords.txt").load()

# 텍스트 분할기 초기화
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,      # 각 청크의 최대 크기
    chunk_overlap=100    # 청크 간 중복 크기
)

# 문서 분할
texts = text_splitter.split_documents(documents)
print(f"문서가 {len(texts)}개의 청크로 분할되었습니다.")

In [3]:
# Cohere 임베딩 모델을 사용한 벡터 스토어 생성
embeddings = CohereEmbeddings(
    model="embed-multilingual-v3.0"  # 다국어 임베딩 모델 사용
)

# FAISS 벡터 스토어 생성 및 검색기 초기화
vectorstore = FAISS.from_documents(texts, embeddings)

# 검색기 생성 (상위 10개 문서 검색)
retriever = vectorstore.as_retriever(
    search_kwargs={"k": 10}
)

## 5. 기본 검색 수행

재정렬을 적용하기 전, 기본 검색 결과를 확인해봅시다.

In [4]:
# 질의문 설정
query = "Word2Vec에 대해서 알려줘!"

# 기본 검색 수행
docs = retriever.invoke(query)

print(f"\n🔍 쿼리: {query}")
print(f"\n📄 검색된 문서 수: {len(docs)}\n")
print("=" * 100)
print("기본 검색 결과 (재정렬 전):")
print("=" * 100)

# 검색 결과 출력
pretty_print_docs(docs)

## 6. Cohere Reranker를 사용한 재정렬

이제 `ContextualCompressionRetriever`와 `CohereRerank`를 사용하여 검색 결과를 재정렬합니다.

### 재정렬의 장점:
1. **정확도 향상**: 쿼리와 가장 관련성 높은 문서를 상위에 배치
2. **노이즈 감소**: 관련 없는 문서 필터링
3. **효율성**: 후속 처리에 필요한 문서 수 감소

In [5]:
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_cohere import CohereRerank

# Cohere 재정렬 모델 설정
compressor = CohereRerank(
    model="rerank-multilingual-v3.0",  # 다국어 재정렬 모델
    top_n=3  # 상위 3개 문서만 반환 (선택사항)
)

# 문맥 압축 검색기 생성
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,  # 재정렬 모델
    base_retriever=retriever      # 기본 검색기
)

In [6]:
# 재정렬된 문서 검색
compressed_docs = compression_retriever.invoke(query)

print(f"\n🔍 쿼리: {query}")
print(f"\n📄 재정렬 후 문서 수: {len(compressed_docs)}\n")
print("=" * 100)
print("재정렬된 검색 결과 (Cohere Reranker 적용):")
print("=" * 100)

# 재정렬된 결과 출력
pretty_print_docs(compressed_docs)

## 7. 재정렬 전후 비교

재정렬이 검색 품질에 미치는 영향을 비교해봅시다.

In [7]:
# 다양한 쿼리로 테스트
test_queries = [
    "Word2Vec에 대해서 알려줘!",
    "임베딩이란 무엇인가?",
    "트랜스포머 모델 설명해줘"
]

for test_query in test_queries:
    print(f"\n{'='*100}")
    print(f"쿼리: {test_query}")
    print(f"{'='*100}")
    
    # 기본 검색
    basic_docs = retriever.invoke(test_query)
    print(f"\n기본 검색 - 상위 3개 문서:")
    for i, doc in enumerate(basic_docs[:3]):
        content_preview = doc.page_content[:100] + "..."
        print(f"  {i+1}. {content_preview}")
    
    # 재정렬 검색
    reranked_docs = compression_retriever.invoke(test_query)
    print(f"\n재정렬 후 - 상위 3개 문서:")
    for i, doc in enumerate(reranked_docs[:3]):
        content_preview = doc.page_content[:100] + "..."
        print(f"  {i+1}. {content_preview}")

## 8. 고급 설정: 재정렬 파라미터 조정

In [8]:
# 다양한 설정으로 재정렬기 생성
advanced_compressor = CohereRerank(
    model="rerank-multilingual-v3.0",
    top_n=5,  # 상위 5개 문서 반환
    # 추가 파라미터 설정 가능
)

# 고급 압축 검색기
advanced_retriever = ContextualCompressionRetriever(
    base_compressor=advanced_compressor,
    base_retriever=retriever
)

# 테스트
advanced_docs = advanced_retriever.invoke("딥러닝과 머신러닝의 차이점")
print(f"고급 설정으로 검색된 문서 수: {len(advanced_docs)}")

## 9. 실습: 다양한 시나리오에서 활용하기

In [9]:
# 시나리오 1: 한국어 쿼리 처리
korean_query = "자연어 처리 기술에 대해 설명해주세요"
korean_docs = compression_retriever.invoke(korean_query)

print("한국어 쿼리 처리 결과:")
print(f"검색된 문서 수: {len(korean_docs)}")
if korean_docs:
    print(f"\n최상위 문서 미리보기:")
    print(korean_docs[0].page_content[:200] + "...")

In [10]:
# 시나리오 2: 영어 쿼리 처리
english_query = "What is embedding in NLP?"
english_docs = compression_retriever.invoke(english_query)

print("\n영어 쿼리 처리 결과:")
print(f"검색된 문서 수: {len(english_docs)}")
if english_docs:
    print(f"\n최상위 문서 미리보기:")
    print(english_docs[0].page_content[:200] + "...")

## 10. 성능 비교 및 평가

In [11]:
import time

def compare_retrieval_methods(query, retriever, compression_retriever):
    """기본 검색과 재정렬 검색의 성능을 비교하는 함수"""
    
    # 기본 검색 시간 측정
    start_time = time.time()
    basic_docs = retriever.invoke(query)
    basic_time = time.time() - start_time
    
    # 재정렬 검색 시간 측정
    start_time = time.time()
    reranked_docs = compression_retriever.invoke(query)
    rerank_time = time.time() - start_time
    
    print(f"쿼리: {query}")
    print(f"\n기본 검색:")
    print(f"  - 소요 시간: {basic_time:.3f}초")
    print(f"  - 검색된 문서 수: {len(basic_docs)}")
    
    print(f"\n재정렬 검색:")
    print(f"  - 소요 시간: {rerank_time:.3f}초")
    print(f"  - 검색된 문서 수: {len(reranked_docs)}")
    print(f"  - 시간 차이: +{rerank_time - basic_time:.3f}초")
    
    return basic_docs, reranked_docs

# 성능 비교 실행
basic, reranked = compare_retrieval_methods(
    "Word2Vec 모델의 원리",
    retriever,
    compression_retriever
)

## 11. 요약 및 결론

### 학습한 내용:
1. **Cohere 임베딩 모델**: 다국어 텍스트를 벡터로 변환
2. **Cohere Reranker**: 검색 결과를 재정렬하여 품질 향상
3. **ContextualCompressionRetriever**: 기본 검색기와 재정렬기를 결합

### 재정렬의 장점:
- ✅ 더 정확한 검색 결과
- ✅ 쿼리와의 관련성 향상
- ✅ 노이즈 감소

### 재정렬의 단점:
- ⚠️ 추가적인 API 호출로 인한 지연
- ⚠️ 추가 비용 발생 가능

### 사용 시나리오:
- 정확도가 중요한 검색 시스템
- RAG (Retrieval-Augmented Generation) 파이프라인
- 다국어 문서 검색 시스템
- 엔터프라이즈 검색 솔루션

## 12. 추가 학습 자료

- [Cohere 공식 문서](https://docs.cohere.com/)
- [LangChain ContextualCompressionRetriever 문서](https://python.langchain.com/docs/modules/data_connection/retrievers/contextual_compression/)
- [Cohere Rerank API 문서](https://docs.cohere.com/docs/reranking)
- [FAISS 벡터 데이터베이스 문서](https://github.com/facebookresearch/faiss)